<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ebe5c582df7f5802a59f3dba1e4e7b35f287aaa70bba830a5ec85381fe656d13
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-09 11:46:30
-------------------
qualified stocks: 90
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  5.27 L
Current:  1.40 C
-------------------
Today PnL: 43.98 K (0.31%)
Current PnL: -17.72 L (-11.92%)
CY Booked + Current PnL: -7.86 L (-5.29%)
-------------------
Total profit:  2.86 L
Total loss:  -20.58 L
-------------------
Total Booked + Current PnL: 19.12 L (15.71%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.03%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.01 L (56.34%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.89%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DIXON,18940.15,-28.26,70.0,H-MC,8.49,143480.0,28544.0,8035.0,-0.39,24.83,5.60,31.83,81.0,3.55,1.06,44.67,X40N,ATH,IT
5,APOLLOHOSP,8285.00,-13.64,60.0,H-LC,10.80,172524.0,16347.0,9748.0,0.71,10.47,5.65,16.71,22.0,1.68,1.28,29.73,X40N,BTT,HEALTHCARE
80,TTKPRESTIG,770.00,105.98,64.0,M-SC,5.88,88720.0,-12057.0,12155.0,0.74,-11.96,13.70,0.09,245.0,-0.99,0.66,16.56,OX40N,NTT,DURABLES
65,SBIN,863.00,-13.25,48.0,M-LC,4.29,214279.0,12800.0,14421.0,-0.02,6.35,6.73,13.51,60.0,0.89,1.59,19.75,XY25,NTT,BANKS
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.92,190260.0,39347.0,17961.0,-0.13,26.07,9.44,37.97,93.0,2.19,1.41,57.97,AR,ATH,AUTO


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,41.94,62.0,M-MC,8.02,224015.0,-947.0,168594.0,-1.03,-0.42,75.26,74.53,192.0,-0.01,1.66,31.70,XY24,BTT,STEEL
4,ANGELONE,3033.00,8.67,41.0,H-SC,6.48,192577.0,1571.0,56136.0,0.32,0.82,29.15,30.21,157.0,0.03,1.43,21.88,X40N,NTT,FINANCE
37,ICICIGI,2260.25,-21.04,40.0,H-MC,3.72,135783.0,-350.0,31474.0,0.10,-0.26,23.18,22.86,91.0,-0.01,1.01,12.32,X40,ATH,INSURANCE
54,PGHH,17973.08,-30.55,54.0,H-MC,4.79,201000.0,180.0,68601.0,0.93,0.09,34.13,34.25,80.0,0.00,1.49,5.29,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,-1.94,37.0,H-LC,10.17,223788.0,-18136.0,81414.0,1.60,-7.50,36.38,26.15,8.0,-0.22,1.66,5.75,X40,ATH,IT
86,VBL,671.64,-15.52,40.0,H-LC,6.65,304007.0,-11835.0,124491.0,0.52,-3.75,40.95,35.67,5.0,-0.10,2.25,9.59,X40N,ATH,FMCG
46,JIOFIN,387.00,-1.47,42.0,H-LC,13.89,217840.0,434.0,53066.0,0.03,0.20,24.36,24.61,48.0,0.01,1.61,55.13,XY24,BTT,FINANCE
77,TCS,4998.00,-28.04,43.0,H-LC,14.19,286879.0,-59077.0,182943.0,1.09,-17.08,63.77,35.80,1.0,-0.32,2.13,1.63,X40,BTT,IT
83,UNITDSPR,1644.00,-14.66,44.0,H-LC,8.35,171745.0,-10014.0,45255.0,0.39,-5.51,26.35,19.39,86.0,-0.22,1.27,2.19,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LTIM,7230.20,-6.94,47.0,H-LC,1.63,230760.0,-15391.0,94589.0,0.77,-6.25,40.99,32.18,16.0,-0.16,1.71,27.54,X200,ATH,IT
53,NESTLEIND,1377.00,-9.37,57.0,H-LC,2.51,279045.0,13619.0,43168.0,0.40,5.13,15.47,21.40,11.0,0.32,2.07,12.84,XY25,NTT,FMCG
44,ITC,452.00,-37.92,51.0,H-LC,3.22,200368.0,230.0,20658.0,0.54,0.11,10.31,10.44,4.0,0.01,1.48,5.68,X40,NTT,FMCG
20,CIPLA,1795.00,-20.81,46.0,H-LC,5.72,211144.0,6644.0,34775.0,-0.79,3.25,16.47,20.25,10.0,0.19,1.56,12.27,X40N,BTT,PHARMA
86,VBL,671.64,-15.52,40.0,H-LC,6.65,304007.0,-11835.0,124491.0,0.52,-3.75,40.95,35.67,5.0,-0.10,2.25,9.59,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,60.0,100.70,67.0,H-SC,12.85,127062.0,-15417.0,34942.0,6.54,-10.82,27.50,13.70,163.0,-0.44,0.94,49.97,OX40N,NTT,BANKS
84,VAIBHAVGBL,521.0,57.58,66.0,H-SC,10.99,105267.0,-46724.0,120847.0,6.23,-30.74,114.80,48.77,125.0,-0.39,0.78,22.52,XR,NTT,JEWELLERY
85,VALIANTORG,838.0,-338.30,66.0,H-SC,8.91,101296.0,-31990.0,116582.0,4.56,-24.00,115.09,63.47,139.0,-0.27,0.75,69.80,XR,NTT,CHEMICALS
42,INFY,2275.0,-18.16,52.0,H-LC,10.57,318840.0,5952.0,165733.0,4.47,1.90,51.98,54.87,3.0,0.04,2.36,8.68,X40,BTT,IT
78,TITAGARH,1548.0,-6.26,58.0,H-SC,7.84,153720.0,-38295.0,117181.0,3.95,-19.94,76.23,41.08,158.0,-0.33,1.14,30.26,XY24,NTT,ENGINEERING


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3748.65,65.0,M-SC,7.78,119900.0,-27468.0,79290.0,-5.07,-18.64,66.13,35.16,236.0,-0.35,0.89,22.47,XY24,NTT,AUTO
67,SHALBY,327.0,1061.08,70.0,M-SC,2.84,157831.0,-23312.0,68783.0,-2.81,-12.87,43.58,25.10,235.0,-0.34,1.17,26.49,XY24,NTT,HEALTHCARE
22,COFFEEDAY,80.0,-37.63,61.0,L-SC,40.50,91238.0,-22311.0,62361.0,-2.64,-19.65,68.35,35.27,268.0,-0.36,0.68,123.31,XR,NTT,HOTELS
71,STARHEALTH,761.0,8.04,50.0,H-SC,3.73,160446.0,-39770.0,115794.0,-2.13,-19.86,72.17,37.97,144.0,-0.34,1.19,28.83,XY24,NTT,INSURANCE
14,BERGEPAINT,680.0,-19.75,43.0,H-MC,7.37,215683.0,-11656.0,59723.0,-1.32,-5.13,27.69,21.14,106.0,-0.20,1.60,21.57,XY24,NTT,PAINTS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,60.0,100.70,67.0,H-SC,12.85,127062.0,-15417.0,34942.0,6.54,-10.82,27.50,13.70,163.0,-0.44,0.94,49.97,OX40N,NTT,BANKS
80,TTKPRESTIG,770.0,105.98,64.0,M-SC,5.88,88720.0,-12057.0,12155.0,0.74,-11.96,13.70,0.09,245.0,-0.99,0.66,16.56,OX40N,NTT,DURABLES
47,KANSAINER,340.0,-67.61,62.0,H-SC,2.60,223848.0,-45819.0,82152.0,2.22,-16.99,36.70,13.47,138.0,-0.56,1.66,13.22,XY24,NTT,PAINTS
40,INDIGOPNTS,1408.0,114.29,49.0,M-SC,3.24,140628.0,-33931.0,33962.0,-0.66,-19.44,24.15,0.02,221.0,-1.00,1.04,22.32,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.41,114156.0,-29647.0,56393.0,-0.67,-20.62,49.40,18.60,149.0,-0.53,0.85,25.97,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,4998.00,-28.04,43.0,H-LC,14.19,286879.0,-59077.0,182943.0,1.09,-17.08,63.77,35.80,1.0,-0.32,2.13,1.63,X40,BTT,IT
42,INFY,2275.00,-18.16,52.0,H-LC,10.57,318840.0,5952.0,165733.0,4.47,1.90,51.98,54.87,3.0,0.04,2.36,8.68,X40,BTT,IT
44,ITC,452.00,-37.92,51.0,H-LC,3.22,200368.0,230.0,20658.0,0.54,0.11,10.31,10.44,4.0,0.01,1.48,5.68,X40,NTT,FMCG
86,VBL,671.64,-15.52,40.0,H-LC,6.65,304007.0,-11835.0,124491.0,0.52,-3.75,40.95,35.67,5.0,-0.10,2.25,9.59,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,44.0,H-LC,12.76,245813.0,-15806.0,135025.0,0.25,-6.04,54.93,45.57,7.0,-0.12,1.82,4.60,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.92,190260.0,39347.0,17961.0,-0.13,26.07,9.44,37.97,93.0,2.19,1.41,57.97,AR,ATH,AUTO
26,DIXON,18940.15,-28.26,70.0,H-MC,8.49,143480.0,28544.0,8035.0,-0.39,24.83,5.60,31.83,81.0,3.55,1.06,44.67,X40N,ATH,IT
27,DMART,5201.00,-6.78,63.0,H-LC,16.57,232819.0,48338.0,22025.0,0.79,26.20,9.46,38.14,38.0,2.19,1.73,39.58,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.17,53.0,H-MC,3.36,188550.0,23880.0,44083.0,0.07,14.50,23.38,41.28,89.0,0.54,1.40,23.64,X40N,ATH,AC
4,ANGELONE,3033.00,8.67,41.0,H-SC,6.48,192577.0,1571.0,56136.0,0.32,0.82,29.15,30.21,157.0,0.03,1.43,21.88,X40N,NTT,FINANCE
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.92,190260.0,39347.0,17961.0,-0.13,26.07,9.44,37.97,93.0,2.19,1.41,57.97,AR,ATH,AUTO
87,VOLTAS,1918.49,1.58,61.0,H-MC,3.47,213045.0,21303.0,74736.0,0.08,11.11,35.08,50.08,99.0,0.29,1.58,18.66,XY25,ATH,AC
24,DABUR,735.00,-2.76,60.0,H-MC,5.13,211497.0,14469.0,75145.0,-0.71,7.34,35.53,45.49,102.0,0.19,1.57,21.34,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.92,190260.0,39347.0,17961.0,-0.13,26.07,9.44,37.97,93.0,2.19,1.41,57.97,AR,ATH,AUTO
52,NATIONALUM,247.44,-44.49,67.0,H-MC,1.74,110320.0,9816.0,20828.0,-1.14,9.77,18.88,30.49,79.0,0.47,0.82,45.34,MH,ATH,METALS
88,WHIRLPOOL,2270.00,-39.02,64.0,M-SC,3.10,103012.0,11514.0,67236.0,0.77,12.58,65.27,86.07,223.0,0.17,0.76,49.59,XR,NTT,DURABLES
39,INDIAMART,4850.92,-50.91,51.0,H-SC,12.10,137058.0,13722.0,120035.0,0.44,11.13,87.58,108.45,119.0,0.11,1.02,35.97,AR,ATH,MISC
89,WIPRO,420.00,-11.98,51.0,M-LC,6.75,154448.0,3503.0,105951.0,2.72,2.32,68.60,72.51,53.0,0.03,1.14,8.48,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6711.11,51.0,L-SC,6.19,74715.0,-19095.0,95717.0,1.15,-20.36,128.11,81.67,269.0,-0.20,0.55,44.69,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-6.02,41.0,H-SC,15.00,87462.0,-13501.0,108033.0,-0.07,-13.37,123.52,93.63,148.0,-0.12,0.65,30.27,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,61.0,L-SC,40.50,91238.0,-22311.0,62361.0,-2.64,-19.65,68.35,35.27,268.0,-0.36,0.68,123.31,XR,NTT,HOTELS
51,MASFIN,398.61,-15.99,61.0,H-SC,12.96,95595.0,-2385.0,23985.0,2.67,-2.43,25.09,22.05,152.0,-0.10,0.71,38.95,XR,ATH,FINANCE
85,VALIANTORG,838.00,-338.30,66.0,H-SC,8.91,101296.0,-31990.0,116582.0,4.56,-24.00,115.09,63.47,139.0,-0.27,0.75,69.80,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,61.0,L-SC,40.50,91238.0,-22311.0,62361.0,-2.64,-19.65,68.35,35.27,268.0,-0.36,0.68,123.31,XR,NTT,HOTELS
8,ATULAUTO,844.00,3748.65,65.0,M-SC,7.78,119900.0,-27468.0,79290.0,-5.07,-18.64,66.13,35.16,236.0,-0.35,0.89,22.47,XY24,NTT,AUTO
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.92,190260.0,39347.0,17961.0,-0.13,26.07,9.44,37.97,93.0,2.19,1.41,57.97,AR,ATH,AUTO
63,SAMMAANCAP,326.00,-171.59,64.0,M-SC,4.60,81960.0,-20250.0,113638.0,-0.81,-19.81,138.65,91.37,208.0,-0.18,0.61,33.05,XY25,NTT,FINANCE
85,VALIANTORG,838.00,-338.30,66.0,H-SC,8.91,101296.0,-31990.0,116582.0,4.56,-24.00,115.09,63.47,139.0,-0.27,0.75,69.80,XR,NTT,CHEMICALS


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.48
1,25,42.97
2,50,73.14


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.21
LC    35.53
MC    25.30
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.54
X40N     15.40
X40      15.36
XY25      9.88
AR        9.35
XR        9.17
OX40N     7.44
MH        1.72
X200      1.71
X5K       1.40
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.34
H-SC    23.97
H-MC    22.32
M-SC    13.63
M-LC     7.13
M-MC     2.66
L-SC     1.61
L-LC     1.06
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.56,-0.77,35.35
IT,13.29,-15.12,72.19
FINANCE,8.72,-18.77,61.48
BANKS,7.36,-14.93,63.73
PAINTS,5.96,-14.91,40.63
MISC,5.80,-17.59,85.00
ELECTRICAL,5.60,-9.03,69.12
AUTO,4.81,-4.19,49.52
AC,3.70,2.92,29.96


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2749637.0,22
AR,1217048.0,10
XR,1057364.0,13
X40,895155.0,10
X40N,602608.0,12
OX40N,454450.0,10
XY25,418521.0,7
SR,198964.0,2
X5K,129287.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2790007.0,28
H-LC,1431026.0,16
M-SC,1405849.0,17
H-MC,1194386.0,17
M-LC,417236.0,5
M-MC,318813.0,2
L-SC,243448.0,3
L-MC,59972.0,1
L-LC,40629.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,925569.0,6
M-SC,XY24,819721.0,7
H-SC,AR,661510.0,5
H-LC,X40,601366.0,5
H-SC,XR,498440.0,6
H-MC,XY24,464891.0,4
H-LC,AR,377590.0,2
M-MC,XY24,318813.0,2
H-SC,X40N,248536.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
